# ChEMBL - Fetching Data

First, we import the ChEMBL webresource library and then pandas to make the conversion into a dataframe

In [1]:
from chembl_webresource_client.new_client import new_client
from rdkit.Chem import PandasTools
import pandas as pd
import re

We can use the `.only()` method to filter the information we want to retrieve:
* indication_class
* first_approval
* inorganic_flag
* molecule_structures
* molecule_type
* natural_product
* oral
* parenteral
* polymer_flag
* pref_name
* prodrug
* therapeutic_flag
* topical
* withdrawn_flag

In [2]:
molecule = new_client.molecule
approved_drugs = molecule.filter(max_phase=4).order_by('molecule_properties__mw_freebase').only(
    'indication_class',
    'first_approval',
    'inorganic_flag',
    'molecule_structures',
    'molecule_type',
    'natural_product',
    'oral',
    'parenteral',
    'polymer_flag',
    'pref_name',
    'prodrug',
    'therapeutic_flag',
    'topical',
    'withdrawn_flag'
)

In [6]:
approved_drugs_df = pd.DataFrame.from_dict(approved_drugs)
approved_drugs_df.head(5)

KeyboardInterrupt: 

In [7]:
molecule = new_client.molecule
mols = molecule.filter(pref_name__iexact='aspirin')
mols

HttpApplicationError: Error for url https://www.ebi.ac.uk/chembl/api/data/molecule.json, server response: <!doctype html>
<html lang="en" class="vf-no-js">
  <head>
    <script>
// Detect if JS is on and swap vf-no-js for vf-js on the html element
(function(H){H.className=H.className.replace(/\bvf-no-js\b/,'vf-js')})(document.documentElement);
</script>

    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <!-- <link rel="stylesheet" media="all" href="/css/styles.css?" /> -->
    <title>Error: 500 | EMBLâs European Bionformatics Institute</title>



    <link rel="icon" type="image/x-icon"
  href="https://ebi.emblstatic.net/web_guidelines/EBI-Framework/v1.4/images/logos/EMBL-EBI/favicons/favicon.ico" />
<link rel="icon" type="image/png"
  href="https://ebi.emblstatic.net/web_guidelines/EBI-Framework/v1.4/images/logos/EMBL-EBI/favicons/favicon-32x32.png" />
<link rel="icon" type="image/png" sizes="192Ã192"
  href="https://ebi.embls

## Pre-cleaning

In this step, we drop some missing and irrelevant stuff and reorder stuff.

Extracting the *canonical_smiles* from the `molecule_structures` variable. We use a lambda function to use the `if/else` condition and apply not found for the absent SMILES.

In [ ]:
# Generating a SMILES column from the 'molecule_structures' variable
approved_drugs_df['SMILES'] = approved_drugs_df['molecule_structures'].apply(lambda x: x['canonical_smiles'] if x != None else 'not found')

Next we do the following pre-cleaning steps:
* Dropping the `molecule_structures` variable as the SMILES are already stored in the `SMILES` variable created in the previous steps;
* Filter only the *Small molecules* on the `molecule_type` variable;
* Remove strictly *inorganic* and *polymers*;
* Remove SMILES not found;
* Remove radioactive, gases and diluent;
* Only `therapeutic_flag` == True;
* Remove structures without approval date;

In [ ]:
# Filtering the structures:

# Dopping the 'molecule_structure' variable because we only want canonical_smiles
approved_drugs_df.drop('molecule_structures',axis=1, inplace=True)

# We want only 'Small molecule', don't want inorganic or polymer stuff
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['molecule_type'] == 'Small molecule']
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['inorganic_flag'] == 0]
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['polymer_flag'] == 0]

# We don't want SMILES not found
approved_drugs_df = approved_drugs_df.loc[approved_drugs_df['SMILES'] != 'not found']

# We don't want indication_class containing the word "gases" or "diluent" or "radioactive"
approved_drugs_df = approved_drugs_df[~approved_drugs_df['indication_class'].str.contains("radioactive|gases|diluent", flags=re.IGNORECASE, regex=True, na=False)]

# with terapeutic_flag == True
approved_drugs_df = approved_drugs_df[approved_drugs_df['therapeutic_flag'] == True]

# Remove the undated first approvals
approved_drugs_df = approved_drugs_df[~approved_drugs_df['first_approval'].isna()].reset_index(drop=True)

Now we can do some tweaking into our dataframe:
* **Rounding** the number of approval date (just for aesthetic)
* **Reordering** the dataframe into a more logic variable sequence

In [ ]:
#Rounding the datetime
approved_drugs_df['first_approval'] = approved_drugs_df['first_approval'].apply(int)

# Reordering the dataframe
approved_drugs_df = approved_drugs_df[['pref_name','SMILES','first_approval','polymer_flag','indication_class','withdrawn_flag','inorganic_flag','polymer_flag','therapeutic_flag','natural_product','oral', 'parenteral', 'topical']]

print("\n", approved_drugs_df.columns, "\n")
print(f"The dataset has the shape {approved_drugs_df.shape}")
approved_drugs_df

Now we're left with the following problems:
* Extremely small structures such as Nitric Oxide, Hydrogen Peroxide that cannot be used to derive any rules. (Maybe this can be solved by applying some MW filter)
* Repeated SMILES (salt and conjugated) - Maybe this can be solved by keeping the largest fragment and dropping x*plicated SMILES.
* Extremely large structures such as OMEGA-3-Carboxylic Acids (Apply some MW filter? Maybe)


Solving the first problem (extremely small structures). We calculate the mol_format 

In [ ]:
# Adding the column into the mol_format
PandasTools.AddMoleculeColumnToFrame(approved_drugs_df, 'SMILES', 'mol_format')